# 🚀 Week 10 — Advanced SQL and Database Integration

```
╔══════════════════════════════════════════════════════════════════╗
║  ADVANCED SQL: Performance, Integration, and Best Practices      ║
║                                                                  ║
║  "The difference between a junior and senior developer is       ║
║   knowing when to use the database vs. Python for operations."  ║
║                                                                  ║
║  This session covers:                                            ║
║  • Complex JOINs and subqueries                                 ║
║  • Performance optimization with indexes                        ║
║  • Clean Python integration patterns                            ║
║  • Repository pattern for data access                           ║
╚══════════════════════════════════════════════════════════════════╝
```

### 🔑 From JavaScript to Python:

| Concept | JavaScript (ORM) | Python (sqlite3) |
|---------|------------------|------------------|
| **Query Builder** | `Model.findAll({ where: ... })` | `cursor.execute(sql, params)` |
| **Aggregation** | `.aggregate('sum', 'total')` | `SUM(total)` in SQL |
| **Relationships** | `include: [Model]` | `JOIN ... ON ...` |
| **Transactions** | `sequelize.transaction()` | Context managers |

> 💡 **Key Insight:** SQL is a declarative language - tell it WHAT you want, not HOW to get it!

## 📋 Learning Objectives

By the end of this session, you will be able to:

| Objective | Skills |
|-----------|--------|
| 🔗 **Write Complex Queries** | Subqueries, CTEs, window functions |
| ⚡ **Optimize Performance** | Indexes, query plans, batch operations |
| 🐍 **Integrate with Python** | Repository pattern, type hints, dataclasses |
| 🔒 **Build Safe APIs** | Parameterized queries, input validation |

### Code Evolution Path:

```
🚫 Bad Practice    →  SQL injection, no error handling
🌱 Novice          →  Basic parameterized queries
🔧 Intermediate    →  Context managers, row_factory
✨ Pythonic        →  Repository pattern, type safety
```

---

## 🛠️ Setup: E-Commerce Database

We'll work with a more realistic e-commerce database schema:

```
┌──────────────┐       ┌──────────────┐       ┌──────────────┐
│    users     │       │   orders     │       │  order_items │
├──────────────┤       ├──────────────┤       ├──────────────┤
│ id (PK)      │───┐   │ id (PK)      │───┐   │ id (PK)      │
│ name         │   └──>│ user_id (FK) │   └──>│ order_id (FK)│
│ email        │       │ order_date   │       │ product_name │
│ created_at   │       │ status       │       │ quantity     │
└──────────────┘       └──────────────┘       │ price        │
                                              └──────────────┘
```

In [ ]:
# ╔══════════════════════════════════════════════════════════════════╗
# ║  🛠️ DATABASE SETUP: E-Commerce Schema                            ║
# ╚══════════════════════════════════════════════════════════════════╝
# Create a realistic e-commerce database for our examples

import sqlite3
from datetime import datetime, timedelta
import random

# Create in-memory database
conn = sqlite3.connect(':memory:')
conn.row_factory = sqlite3.Row  # Enable named column access
cur = conn.cursor()

# Enable foreign keys
cur.execute('PRAGMA foreign_keys = ON')

# ═══════════════════════════════════════════════════════════════════
# Create Schema with Relationships
# ═══════════════════════════════════════════════════════════════════

cur.executescript('''
    -- Users table
    CREATE TABLE users (
        id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        email TEXT UNIQUE NOT NULL,
        created_at TEXT DEFAULT CURRENT_TIMESTAMP
    );
    
    -- Orders table
    CREATE TABLE orders (
        id INTEGER PRIMARY KEY,
        user_id INTEGER NOT NULL,
        order_date TEXT DEFAULT CURRENT_DATE,
        status TEXT DEFAULT 'pending',
        FOREIGN KEY (user_id) REFERENCES users(id)
    );
    
    -- Order items (line items)
    CREATE TABLE order_items (
        id INTEGER PRIMARY KEY,
        order_id INTEGER NOT NULL,
        product_name TEXT NOT NULL,
        quantity INTEGER NOT NULL DEFAULT 1,
        price REAL NOT NULL,
        FOREIGN KEY (order_id) REFERENCES orders(id)
    );
''')

# ═══════════════════════════════════════════════════════════════════
# Populate with Sample Data
# ═══════════════════════════════════════════════════════════════════

# Users
users = [
    ('Ana Garcia', 'ana@email.com'),
    ('Bob Smith', 'bob@email.com'),
    ('Cara Jones', 'cara@email.com'),
    ('Dan Wilson', 'dan@email.com'),
]
cur.executemany('INSERT INTO users (name, email) VALUES (?, ?)', users)

# Orders with dates
orders = [
    (1, '2024-01-15', 'completed'),
    (1, '2024-02-20', 'completed'),
    (2, '2024-01-10', 'completed'),
    (2, '2024-03-05', 'pending'),
    (3, '2024-02-28', 'completed'),
    (3, '2024-03-01', 'shipped'),
    # Dan has no orders yet
]
cur.executemany('INSERT INTO orders (user_id, order_date, status) VALUES (?, ?, ?)', orders)

# Order items (multiple items per order)
order_items = [
    (1, 'Laptop', 1, 999.99),
    (1, 'Mouse', 2, 29.99),
    (2, 'Keyboard', 1, 149.99),
    (2, 'Monitor', 1, 399.99),
    (3, 'Headphones', 1, 199.99),
    (4, 'USB Cable', 3, 9.99),
    (5, 'Webcam', 1, 79.99),
    (6, 'Laptop Stand', 1, 49.99),
    (6, 'Mouse Pad', 2, 19.99),
]
cur.executemany(
    'INSERT INTO order_items (order_id, product_name, quantity, price) VALUES (?, ?, ?, ?)',
    order_items
)

conn.commit()

# Verify setup
print("✅ Database Setup Complete!")
print("\n📊 Database Statistics:")
for table in ['users', 'orders', 'order_items']:
    cur.execute(f'SELECT COUNT(*) FROM {table}')
    print(f"  {table}: {cur.fetchone()[0]} records")

---

## 🔗 Section 1: Advanced Aggregates and JOINs

```
╔══════════════════════════════════════════════════════════════════╗
║  Understanding Multi-Table Operations                            ║
║                                                                  ║
║  SQL Power: Let the database do the heavy lifting!              ║
║                                                                  ║
║  • Multiple JOINs - Connect many tables                         ║
║  • Subqueries - Query within a query                            ║
║  • CTEs - Common Table Expressions (WITH clause)                ║
║  • Window Functions - Analytics over partitions                 ║
╚══════════════════════════════════════════════════════════════════╝
```

### The Query Execution Order:

```
1. FROM       - Get the tables
2. WHERE      - Filter rows
3. GROUP BY   - Create groups
4. HAVING     - Filter groups
5. SELECT     - Choose columns
6. ORDER BY   - Sort results
7. LIMIT      - Limit output
```

In [ ]:
# ╔══════════════════════════════════════════════════════════════════╗
# ║  🚫 BAD PRACTICE: N+1 Query Problem                              ║
# ╚══════════════════════════════════════════════════════════════════╝
# The classic mistake: querying in loops instead of using JOINs

import sqlite3

# ❌ Bad: N+1 Query Problem
# One query to get users + N queries to get their orders
print("❌ BAD: N+1 Query Problem (very slow!)\n")

# First query: get all users
cur.execute('SELECT id, name FROM users')
users = cur.fetchall()

results = []
for user in users:
    # N more queries: one per user! 😱
    cur.execute('''
        SELECT SUM(oi.quantity * oi.price) as total
        FROM orders o
        JOIN order_items oi ON o.id = oi.order_id
        WHERE o.user_id = ?
    ''', (user['id'],))
    
    total = cur.fetchone()['total'] or 0
    results.append((user['name'], total))
    print(f"  Query for {user['name']}...")  # Each is a separate DB call!

print("\nResults (after 5 separate queries!):")
for name, total in results:
    print(f"  {name}: ${total:.2f}")

# This approach:
# - Makes 1 + N database calls (N = number of users)
# - Gets slower as data grows
# - Wastes network/processing resources
# - Doesn't use database optimization

In [ ]:
# ╔══════════════════════════════════════════════════════════════════╗
# ║  🌱 NOVICE: Single Query with JOINs                              ║
# ╚══════════════════════════════════════════════════════════════════╝
# Use JOINs to get all data in one query

print("🌱 NOVICE: Single Query with Multiple JOINs\n")

# ✅ Better: One query to rule them all!
query = '''
    SELECT 
        u.name,
        SUM(oi.quantity * oi.price) as total_spend
    FROM users u
    LEFT JOIN orders o ON u.id = o.user_id
    LEFT JOIN order_items oi ON o.id = oi.order_id
    GROUP BY u.id
    ORDER BY total_spend DESC
'''

cur.execute(query)
results = cur.fetchall()

print("Results (in ONE query!):")
for row in results:
    total = row['total_spend'] or 0
    print(f"  {row['name']}: ${total:.2f}")

# Benefits:
# - Single database round-trip
# - Database optimizer handles the work
# - Scales well with data growth
# - Uses indexes efficiently

In [ ]:
# ╔══════════════════════════════════════════════════════════════════╗
# ║  🔧 INTERMEDIATE: Subqueries and CTEs                            ║
# ╚══════════════════════════════════════════════════════════════════╝
# Using subqueries and Common Table Expressions for complex analysis

print("🔧 INTERMEDIATE: Subqueries and CTEs\n")

# ═══════════════════════════════════════════════════════════════════
# Subquery Example: Users who spent more than average
# ═══════════════════════════════════════════════════════════════════

print("📊 Users Who Spent More Than Average:")
query_subquery = '''
    SELECT 
        u.name,
        SUM(oi.quantity * oi.price) as total_spend
    FROM users u
    JOIN orders o ON u.id = o.user_id
    JOIN order_items oi ON o.id = oi.order_id
    GROUP BY u.id
    HAVING SUM(oi.quantity * oi.price) > (
        -- Subquery: Calculate average spend
        SELECT AVG(user_total) FROM (
            SELECT SUM(oi2.quantity * oi2.price) as user_total
            FROM orders o2
            JOIN order_items oi2 ON o2.id = oi2.order_id
            GROUP BY o2.user_id
        )
    )
    ORDER BY total_spend DESC
'''

cur.execute(query_subquery)
for row in cur.fetchall():
    print(f"  {row['name']}: ${row['total_spend']:.2f}")

# ═══════════════════════════════════════════════════════════════════
# CTE Example: Step-by-step analysis (more readable!)
# ═══════════════════════════════════════════════════════════════════

print("\n📊 Using CTE (Common Table Expression):")
query_cte = '''
    -- CTE: Calculate user totals first
    WITH user_totals AS (
        SELECT 
            u.id,
            u.name,
            SUM(oi.quantity * oi.price) as total_spend,
            COUNT(DISTINCT o.id) as order_count
        FROM users u
        LEFT JOIN orders o ON u.id = o.user_id
        LEFT JOIN order_items oi ON o.id = oi.order_id
        GROUP BY u.id
    ),
    -- CTE: Calculate average
    average_spend AS (
        SELECT AVG(total_spend) as avg_total FROM user_totals
    )
    -- Main query: Compare each user to average
    SELECT 
        ut.name,
        ut.total_spend,
        ut.order_count,
        (SELECT avg_total FROM average_spend) as avg_spend,
        CASE 
            WHEN ut.total_spend > (SELECT avg_total FROM average_spend) THEN '⬆️ Above'
            WHEN ut.total_spend < (SELECT avg_total FROM average_spend) THEN '⬇️ Below'
            ELSE '➡️ At Average'
        END as vs_average
    FROM user_totals ut
    ORDER BY ut.total_spend DESC
'''

cur.execute(query_cte)
results = cur.fetchall()

# Get average for display
avg = results[0]['avg_spend'] if results else 0
print(f"  Average spend: ${avg:.2f}\n")

for row in results:
    total = row['total_spend'] or 0
    print(f"  {row['name']}: ${total:.2f} ({row['order_count']} orders) {row['vs_average']}")

In [ ]:
# ╔══════════════════════════════════════════════════════════════════╗
# ║  ✨ PYTHONIC: Advanced Analytics Queries                         ║
# ╚══════════════════════════════════════════════════════════════════╝
# Production-ready analytics with type safety and clean patterns

from typing import List, Dict, Optional, NamedTuple
from dataclasses import dataclass
from enum import Enum

# ═══════════════════════════════════════════════════════════════════
# ✨ Type-Safe Result Classes
# ═══════════════════════════════════════════════════════════════════

class CustomerSegment(Enum):
    """Customer segmentation based on spending."""
    VIP = "VIP"           # Top 20%
    REGULAR = "Regular"   # Middle 60%
    NEW = "New"           # Bottom 20% or no orders

@dataclass
class CustomerAnalytics:
    """Analytics data for a customer."""
    user_id: int
    name: str
    email: str
    total_orders: int
    total_items: int
    total_spend: float
    avg_order_value: float
    segment: CustomerSegment

# ═══════════════════════════════════════════════════════════════════
# ✨ Analytics Query Builder
# ═══════════════════════════════════════════════════════════════════

class AnalyticsService:
    """Service for running analytics queries."""
    
    CUSTOMER_ANALYTICS_QUERY = '''
        WITH customer_stats AS (
            SELECT 
                u.id,
                u.name,
                u.email,
                COUNT(DISTINCT o.id) as total_orders,
                COALESCE(SUM(oi.quantity), 0) as total_items,
                COALESCE(SUM(oi.quantity * oi.price), 0) as total_spend
            FROM users u
            LEFT JOIN orders o ON u.id = o.user_id
            LEFT JOIN order_items oi ON o.id = oi.order_id
            GROUP BY u.id
        ),
        percentiles AS (
            SELECT 
                total_spend,
                NTILE(5) OVER (ORDER BY total_spend) as quintile
            FROM customer_stats
        )
        SELECT 
            cs.*,
            CASE 
                WHEN cs.total_orders = 0 THEN 0
                ELSE cs.total_spend / cs.total_orders 
            END as avg_order_value,
            CASE
                WHEN cs.total_spend = 0 THEN 'New'
                WHEN cs.total_spend >= (
                    SELECT MIN(total_spend) FROM percentiles WHERE quintile = 5
                ) THEN 'VIP'
                ELSE 'Regular'
            END as segment
        FROM customer_stats cs
        ORDER BY cs.total_spend DESC
    '''
    
    TOP_PRODUCTS_QUERY = '''
        SELECT 
            oi.product_name,
            SUM(oi.quantity) as units_sold,
            SUM(oi.quantity * oi.price) as revenue,
            COUNT(DISTINCT o.user_id) as unique_customers
        FROM order_items oi
        JOIN orders o ON oi.order_id = o.id
        GROUP BY oi.product_name
        ORDER BY revenue DESC
        LIMIT ?
    '''
    
    MONTHLY_REVENUE_QUERY = '''
        SELECT 
            strftime('%Y-%m', o.order_date) as month,
            COUNT(DISTINCT o.id) as order_count,
            COUNT(DISTINCT o.user_id) as active_customers,
            SUM(oi.quantity * oi.price) as revenue
        FROM orders o
        JOIN order_items oi ON o.id = oi.order_id
        GROUP BY month
        ORDER BY month
    '''
    
    def __init__(self, connection: sqlite3.Connection):
        self.conn = connection
        self.conn.row_factory = sqlite3.Row
    
    def get_customer_analytics(self) -> List[CustomerAnalytics]:
        """Get analytics for all customers."""
        cursor = self.conn.execute(self.CUSTOMER_ANALYTICS_QUERY)
        
        return [
            CustomerAnalytics(
                user_id=row['id'],
                name=row['name'],
                email=row['email'],
                total_orders=row['total_orders'],
                total_items=row['total_items'],
                total_spend=row['total_spend'],
                avg_order_value=row['avg_order_value'],
                segment=CustomerSegment(row['segment'])
            )
            for row in cursor.fetchall()
        ]
    
    def get_top_products(self, limit: int = 5) -> List[Dict]:
        """Get top products by revenue."""
        cursor = self.conn.execute(self.TOP_PRODUCTS_QUERY, (limit,))
        return [dict(row) for row in cursor.fetchall()]
    
    def get_monthly_revenue(self) -> List[Dict]:
        """Get monthly revenue breakdown."""
        cursor = self.conn.execute(self.MONTHLY_REVENUE_QUERY)
        return [dict(row) for row in cursor.fetchall()]

# ═══════════════════════════════════════════════════════════════════
# ✨ Demonstration
# ═══════════════════════════════════════════════════════════════════

print("✨ PYTHONIC: Advanced Analytics\n")

analytics = AnalyticsService(conn)

# Customer Analytics
print("👥 Customer Analytics:")
print("-" * 70)
for customer in analytics.get_customer_analytics():
    segment_icon = "🌟" if customer.segment == CustomerSegment.VIP else "👤"
    print(f"  {segment_icon} {customer.name} [{customer.segment.value}]")
    print(f"     Orders: {customer.total_orders}, Items: {customer.total_items}, "
          f"Total: ${customer.total_spend:.2f}")

# Top Products
print("\n📦 Top Products:")
print("-" * 70)
for product in analytics.get_top_products(5):
    print(f"  {product['product_name']}: ${product['revenue']:.2f} "
          f"({product['units_sold']} sold to {product['unique_customers']} customers)")

# Monthly Revenue
print("\n📅 Monthly Revenue:")
print("-" * 70)
for month in analytics.get_monthly_revenue():
    print(f"  {month['month']}: {month['order_count']} orders, "
          f"${month['revenue']:.2f} revenue")

---

## 🎯 Activity 1: Top Spenders Query

### Your Task:

Return the **top 2 spenders** using SQL only (with LIMIT).

### Requirements:
1. JOIN users, orders, and order_items
2. Calculate total spend per user
3. Order by spend descending
4. LIMIT to top 2

### 💡 Hints:

```sql
SELECT u.name, SUM(oi.quantity * oi.price) as total
FROM users u
JOIN orders o ON ...
JOIN order_items oi ON ...
GROUP BY ...
ORDER BY ... DESC
LIMIT ...
```

In [ ]:
# ╔══════════════════════════════════════════════════════════════════╗
# ║  👩‍💻 YOUR WORKSPACE: Top 2 Spenders                               ║
# ╚══════════════════════════════════════════════════════════════════╝
# Write a query to return only the top 2 spenders

# ═══════════════════════════════════════════════════════════════════
# 📝 YOUR CODE HERE
# ═══════════════════════════════════════════════════════════════════

# TODO: Complete the query
your_query = '''
    SELECT 
        u.name,
        -- Calculate total spend here
    FROM users u
    -- Add JOINs here
    -- Add GROUP BY
    -- Add ORDER BY
    -- Add LIMIT
'''

# Uncomment to test:
# cur.execute(your_query)
# for row in cur.fetchall():
#     print(row)

# ═══════════════════════════════════════════════════════════════════
# 💡 SOLUTION
# ═══════════════════════════════════════════════════════════════════

solution_query = '''
    SELECT 
        u.name,
        SUM(oi.quantity * oi.price) as total_spend
    FROM users u
    JOIN orders o ON u.id = o.user_id
    JOIN order_items oi ON o.id = oi.order_id
    GROUP BY u.id
    ORDER BY total_spend DESC
    LIMIT 2
'''

print("✅ Top 2 Spenders:")
cur.execute(solution_query)
for row in cur.fetchall():
    print(f"  {row['name']}: ${row['total_spend']:.2f}")

---

## ⚡ Section 2: Indexes and Performance

```
╔══════════════════════════════════════════════════════════════════╗
║  Understanding Database Indexes                                  ║
║                                                                  ║
║  "Without an index, the database scans every row."              ║
║                                                                  ║
║  An index is like a book's index:                               ║
║  • Without: Read every page to find "Python"                    ║
║  • With: Look up "Python" → Page 42                             ║
╚══════════════════════════════════════════════════════════════════╝
```

### When to Use Indexes:

| Scenario | Index? | Why |
|----------|--------|-----|
| **WHERE** clauses | ✅ Yes | Speeds up filtering |
| **JOIN** columns | ✅ Yes | Speeds up matching |
| **ORDER BY** columns | ✅ Maybe | Speeds up sorting |
| **Small tables** | ❌ No | Overhead not worth it |
| **Frequently updated** | ⚠️ Careful | Slows down writes |

### Index Types in SQLite:

```sql
-- Single column index
CREATE INDEX idx_user_email ON users(email);

-- Composite index (multiple columns)
CREATE INDEX idx_order_user_date ON orders(user_id, order_date);

-- Unique index (enforces uniqueness)
CREATE UNIQUE INDEX idx_user_email ON users(email);
```

In [ ]:
# ╔══════════════════════════════════════════════════════════════════╗
# ║  ⚡ INDEX DEMONSTRATION: Before and After                        ║
# ╚══════════════════════════════════════════════════════════════════╝
# Understanding how indexes improve query performance

import sqlite3
import time

# Create a larger test database
test_conn = sqlite3.connect(':memory:')
test_cur = test_conn.cursor()

# Create table WITHOUT index
test_cur.execute('''
    CREATE TABLE large_orders (
        id INTEGER PRIMARY KEY,
        user_id INTEGER,
        amount REAL,
        order_date TEXT
    )
''')

# Insert 100,000 rows for testing
print("⏳ Creating test data (100,000 rows)...")
test_data = [(i % 1000, i * 1.5, f'2024-{(i % 12) + 1:02d}-{(i % 28) + 1:02d}') 
             for i in range(100_000)]
test_cur.executemany(
    'INSERT INTO large_orders (user_id, amount, order_date) VALUES (?, ?, ?)',
    test_data
)
test_conn.commit()
print("✅ Test data created!\n")

# ═══════════════════════════════════════════════════════════════════
# Test 1: Query WITHOUT Index
# ═══════════════════════════════════════════════════════════════════

print("🔍 Query WITHOUT Index:")
print("-" * 50)

# Check query plan
test_cur.execute('EXPLAIN QUERY PLAN SELECT * FROM large_orders WHERE user_id = 500')
plan = test_cur.fetchone()
print(f"  Query Plan: {plan[-1]}")

# Time the query
start = time.perf_counter()
for _ in range(100):  # Run 100 times for measurable difference
    test_cur.execute('SELECT COUNT(*) FROM large_orders WHERE user_id = 500')
    test_cur.fetchone()
end = time.perf_counter()
print(f"  Time (100 queries): {(end - start) * 1000:.2f} ms")

# ═══════════════════════════════════════════════════════════════════
# Create Index
# ═══════════════════════════════════════════════════════════════════

print("\n⚡ Creating index on user_id...")
test_cur.execute('CREATE INDEX idx_orders_user ON large_orders(user_id)')
test_conn.commit()
print("✅ Index created!\n")

# ═══════════════════════════════════════════════════════════════════
# Test 2: Query WITH Index
# ═══════════════════════════════════════════════════════════════════

print("🔍 Query WITH Index:")
print("-" * 50)

# Check query plan again
test_cur.execute('EXPLAIN QUERY PLAN SELECT * FROM large_orders WHERE user_id = 500')
plan = test_cur.fetchone()
print(f"  Query Plan: {plan[-1]}")

# Time the query
start = time.perf_counter()
for _ in range(100):
    test_cur.execute('SELECT COUNT(*) FROM large_orders WHERE user_id = 500')
    test_cur.fetchone()
end = time.perf_counter()
print(f"  Time (100 queries): {(end - start) * 1000:.2f} ms")

# ═══════════════════════════════════════════════════════════════════
# Summary
# ═══════════════════════════════════════════════════════════════════

print("\n" + "=" * 50)
print("📊 Summary:")
print("  Without index: Full table scan (SCAN large_orders)")
print("  With index: Index lookup (SEARCH using index)")
print("  Performance improvement: Often 10-100x faster!")

test_conn.close()

---

## 🎯 Activity 2: Index Performance Analysis

### Your Task:

Create a composite index and measure its impact on a query that filters by multiple columns.

### Steps:
1. Create a large test table
2. Run a query filtering by TWO columns (measure time)
3. Create a composite index on both columns
4. Run the same query (measure time)
5. Compare results

### 💡 Hints:

```sql
-- Composite index on multiple columns
CREATE INDEX idx_name ON table(column1, column2);

-- Query that benefits from composite index
SELECT * FROM table WHERE column1 = ? AND column2 = ?;
```

In [ ]:
# ╔══════════════════════════════════════════════════════════════════╗
# ║  👩‍💻 YOUR WORKSPACE: Composite Index Performance                  ║
# ╚══════════════════════════════════════════════════════════════════╝
# Experiment with composite indexes

import sqlite3
import time

# Setup test database
test_conn = sqlite3.connect(':memory:')
test_cur = test_conn.cursor()

test_cur.execute('''
    CREATE TABLE events (
        id INTEGER PRIMARY KEY,
        user_id INTEGER,
        event_type TEXT,
        created_at TEXT
    )
''')

# Insert 50,000 rows
print("⏳ Creating test data...")
event_types = ['click', 'view', 'purchase', 'signup', 'logout']
test_data = [
    (i % 500, event_types[i % 5], f'2024-{(i % 12) + 1:02d}-{(i % 28) + 1:02d}')
    for i in range(50_000)
]
test_cur.executemany(
    'INSERT INTO events (user_id, event_type, created_at) VALUES (?, ?, ?)',
    test_data
)
test_conn.commit()
print("✅ Data created!\n")

# ═══════════════════════════════════════════════════════════════════
# 📝 YOUR CODE HERE: Test composite index performance
# ═══════════════════════════════════════════════════════════════════

# TODO: 
# 1. Time this query (filtering by user_id AND event_type)
query = "SELECT COUNT(*) FROM events WHERE user_id = 100 AND event_type = 'purchase'"

# 2. Create a composite index:
# test_cur.execute('CREATE INDEX idx_user_event ON events(user_id, event_type)')

# 3. Time the query again and compare

# ═══════════════════════════════════════════════════════════════════
# 💡 SOLUTION
# ═══════════════════════════════════════════════════════════════════

print("Before composite index:")
test_cur.execute(f'EXPLAIN QUERY PLAN {query}')
print(f"  Plan: {test_cur.fetchone()[-1]}")

start = time.perf_counter()
for _ in range(100):
    test_cur.execute(query)
    test_cur.fetchone()
before = (time.perf_counter() - start) * 1000
print(f"  Time: {before:.2f} ms (100 queries)")

# Create composite index
test_cur.execute('CREATE INDEX idx_user_event ON events(user_id, event_type)')
test_conn.commit()

print("\nAfter composite index:")
test_cur.execute(f'EXPLAIN QUERY PLAN {query}')
print(f"  Plan: {test_cur.fetchone()[-1]}")

start = time.perf_counter()
for _ in range(100):
    test_cur.execute(query)
    test_cur.fetchone()
after = (time.perf_counter() - start) * 1000
print(f"  Time: {after:.2f} ms (100 queries)")

improvement = ((before - after) / before) * 100 if before > after else 0
print(f"\n📈 Improvement: {improvement:.1f}% faster!")

test_conn.close()

---

## 🐍 Section 3: Clean Python Integration

```
╔══════════════════════════════════════════════════════════════════╗
║  The Repository Pattern                                          ║
║                                                                  ║
║  Separate data access from business logic:                      ║
║                                                                  ║
║  ┌─────────────────┐    ┌─────────────────┐    ┌─────────────┐  ║
║  │   Your Code     │───>│   Repository    │───>│  Database   │  ║
║  │   (Business)    │    │ (Data Access)   │    │  (SQLite)   │  ║
║  └─────────────────┘    └─────────────────┘    └─────────────┘  ║
║                                                                  ║
║  Benefits:                                                       ║
║  • Single responsibility - each layer does one thing            ║
║  • Testability - mock the repository for tests                  ║
║  • Flexibility - swap databases without code changes            ║
╚══════════════════════════════════════════════════════════════════╝
```

In [ ]:
# ╔══════════════════════════════════════════════════════════════════╗
# ║  🚫 BAD PRACTICE: SQL Everywhere                                 ║
# ╚══════════════════════════════════════════════════════════════════╝
# SQL scattered throughout your code

# ❌ Bad: SQL mixed with business logic
def process_order_bad(user_name, items):
    """Process an order - BAD way"""
    # SQL in business logic
    cur.execute('SELECT id FROM users WHERE name = ?', (user_name,))
    user = cur.fetchone()
    if not user:
        # More SQL
        cur.execute('INSERT INTO users (name, email) VALUES (?, ?)', 
                   (user_name, f'{user_name.lower()}@email.com'))
        user_id = cur.lastrowid
    else:
        user_id = user['id']
    
    # Even more SQL
    cur.execute('INSERT INTO orders (user_id) VALUES (?)', (user_id,))
    order_id = cur.lastrowid
    
    # SQL in a loop!
    for item in items:
        cur.execute('''
            INSERT INTO order_items (order_id, product_name, quantity, price)
            VALUES (?, ?, ?, ?)
        ''', (order_id, item['name'], item['qty'], item['price']))
    
    conn.commit()
    return order_id

# Problems:
# - SQL scattered everywhere
# - Hard to test (needs real database)
# - Hard to maintain (change SQL in multiple places)
# - Business logic mixed with data access

print("❌ Bad Practice Example - SQL mixed with business logic")
print("   Problems: Hard to test, maintain, and modify")

In [ ]:
# ╔══════════════════════════════════════════════════════════════════╗
# ║  🌱 NOVICE: Basic Function Wrapper                               ║
# ╚══════════════════════════════════════════════════════════════════╝
# Wrapping SQL in functions

def get_user_spend(name: str) -> float:
    """
    Get total spend for a user.
    
    Args:
        name: User's name
        
    Returns:
        Total amount spent by user
    """
    cur.execute('''
        SELECT SUM(oi.quantity * oi.price) as total
        FROM users u
        JOIN orders o ON u.id = o.user_id
        JOIN order_items oi ON o.id = oi.order_id
        WHERE u.name = ?
    ''', (name,))
    
    row = cur.fetchone()
    return row['total'] if row and row['total'] else 0.0

def get_user_orders(name: str) -> list:
    """Get all orders for a user."""
    cur.execute('''
        SELECT o.id, o.order_date, o.status
        FROM orders o
        JOIN users u ON o.user_id = u.id
        WHERE u.name = ?
        ORDER BY o.order_date DESC
    ''', (name,))
    
    return [dict(row) for row in cur.fetchall()]

# ✅ Better: SQL isolated in functions
print("🌱 NOVICE: SQL wrapped in functions\n")

print(f"Ana's total spend: ${get_user_spend('Ana'):.2f}")
print(f"Ana's orders: {get_user_orders('Ana')}")

In [ ]:
# ╔══════════════════════════════════════════════════════════════════╗
# ║  🔧 INTERMEDIATE: Repository Class                               ║
# ╚══════════════════════════════════════════════════════════════════╝
# Centralized data access with a repository class

from typing import List, Optional, Dict, Any
from contextlib import contextmanager

class UserRepository:
    """Repository for user data access."""
    
    def __init__(self, connection: sqlite3.Connection):
        self.conn = connection
        self.conn.row_factory = sqlite3.Row
    
    def get_by_name(self, name: str) -> Optional[Dict]:
        """Find user by name."""
        cursor = self.conn.execute(
            'SELECT * FROM users WHERE name = ?', (name,)
        )
        row = cursor.fetchone()
        return dict(row) if row else None
    
    def get_by_id(self, user_id: int) -> Optional[Dict]:
        """Find user by ID."""
        cursor = self.conn.execute(
            'SELECT * FROM users WHERE id = ?', (user_id,)
        )
        row = cursor.fetchone()
        return dict(row) if row else None
    
    def create(self, name: str, email: str) -> int:
        """Create a new user and return their ID."""
        cursor = self.conn.execute(
            'INSERT INTO users (name, email) VALUES (?, ?)',
            (name, email)
        )
        self.conn.commit()
        return cursor.lastrowid
    
    def get_spend(self, user_id: int) -> float:
        """Get total spend for a user."""
        cursor = self.conn.execute('''
            SELECT COALESCE(SUM(oi.quantity * oi.price), 0) as total
            FROM orders o
            JOIN order_items oi ON o.id = oi.order_id
            WHERE o.user_id = ?
        ''', (user_id,))
        return cursor.fetchone()['total']

class OrderRepository:
    """Repository for order data access."""
    
    def __init__(self, connection: sqlite3.Connection):
        self.conn = connection
        self.conn.row_factory = sqlite3.Row
    
    def get_by_user(self, user_id: int) -> List[Dict]:
        """Get all orders for a user."""
        cursor = self.conn.execute('''
            SELECT o.*, SUM(oi.quantity * oi.price) as total
            FROM orders o
            LEFT JOIN order_items oi ON o.id = oi.order_id
            WHERE o.user_id = ?
            GROUP BY o.id
            ORDER BY o.order_date DESC
        ''', (user_id,))
        return [dict(row) for row in cursor.fetchall()]
    
    def create_with_items(self, user_id: int, items: List[Dict]) -> int:
        """Create order with items in a transaction."""
        cursor = self.conn.cursor()
        
        # Create order
        cursor.execute(
            'INSERT INTO orders (user_id) VALUES (?)',
            (user_id,)
        )
        order_id = cursor.lastrowid
        
        # Add items
        cursor.executemany(
            '''INSERT INTO order_items (order_id, product_name, quantity, price)
               VALUES (?, ?, ?, ?)''',
            [(order_id, i['name'], i['qty'], i['price']) for i in items]
        )
        
        self.conn.commit()
        return order_id

# ═══════════════════════════════════════════════════════════════════
# 🔧 Demonstration
# ═══════════════════════════════════════════════════════════════════

print("🔧 INTERMEDIATE: Repository Pattern\n")

user_repo = UserRepository(conn)
order_repo = OrderRepository(conn)

# Find user and get their data
user = user_repo.get_by_name('Ana')
if user:
    print(f"User: {user['name']} ({user['email']})")
    print(f"Total Spend: ${user_repo.get_spend(user['id']):.2f}")
    print(f"Orders: {len(order_repo.get_by_user(user['id']))}")

# Create a new order using the repository
new_order_id = order_repo.create_with_items(
    user['id'],
    [{'name': 'New Product', 'qty': 1, 'price': 49.99}]
)
print(f"\n✅ Created order #{new_order_id}")

In [ ]:
# ╔══════════════════════════════════════════════════════════════════╗
# ║  ✨ PYTHONIC: Enterprise Data Access Layer                       ║
# ╚══════════════════════════════════════════════════════════════════╝
# Production-ready data access with type safety and dependency injection

from typing import List, Optional, TypeVar, Generic, Protocol
from dataclasses import dataclass, field
from abc import ABC, abstractmethod
from datetime import datetime
from contextlib import contextmanager

# ═══════════════════════════════════════════════════════════════════
# ✨ Domain Models (Entities)
# ═══════════════════════════════════════════════════════════════════

@dataclass
class User:
    """User domain entity."""
    id: Optional[int]
    name: str
    email: str
    created_at: Optional[str] = None

@dataclass
class OrderItem:
    """Order item domain entity."""
    id: Optional[int]
    order_id: int
    product_name: str
    quantity: int
    price: float
    
    @property
    def total(self) -> float:
        return self.quantity * self.price

@dataclass
class Order:
    """Order domain entity with items."""
    id: Optional[int]
    user_id: int
    order_date: str
    status: str
    items: List[OrderItem] = field(default_factory=list)
    
    @property
    def total(self) -> float:
        return sum(item.total for item in self.items)

# ═══════════════════════════════════════════════════════════════════
# ✨ Repository Interface (Abstract Base)
# ═══════════════════════════════════════════════════════════════════

T = TypeVar('T')

class Repository(ABC, Generic[T]):
    """Abstract base for repositories."""
    
    @abstractmethod
    def get_by_id(self, id: int) -> Optional[T]:
        """Get entity by ID."""
        pass
    
    @abstractmethod
    def save(self, entity: T) -> T:
        """Save entity (create or update)."""
        pass

# ═══════════════════════════════════════════════════════════════════
# ✨ Concrete Repositories with Type Safety
# ═══════════════════════════════════════════════════════════════════

class SQLiteUserRepository(Repository[User]):
    """SQLite implementation of user repository."""
    
    def __init__(self, connection: sqlite3.Connection):
        self.conn = connection
        self.conn.row_factory = sqlite3.Row
    
    def get_by_id(self, user_id: int) -> Optional[User]:
        cursor = self.conn.execute(
            'SELECT * FROM users WHERE id = ?', (user_id,)
        )
        row = cursor.fetchone()
        if not row:
            return None
        return User(
            id=row['id'],
            name=row['name'],
            email=row['email'],
            created_at=row['created_at']
        )
    
    def get_by_email(self, email: str) -> Optional[User]:
        cursor = self.conn.execute(
            'SELECT * FROM users WHERE email = ?', (email,)
        )
        row = cursor.fetchone()
        if not row:
            return None
        return User(
            id=row['id'],
            name=row['name'],
            email=row['email'],
            created_at=row['created_at']
        )
    
    def save(self, user: User) -> User:
        if user.id is None:
            cursor = self.conn.execute(
                'INSERT INTO users (name, email) VALUES (?, ?)',
                (user.name, user.email)
            )
            user.id = cursor.lastrowid
        else:
            self.conn.execute(
                'UPDATE users SET name = ?, email = ? WHERE id = ?',
                (user.name, user.email, user.id)
            )
        self.conn.commit()
        return user
    
    def get_all_with_stats(self) -> List[dict]:
        """Get all users with their order statistics."""
        cursor = self.conn.execute('''
            SELECT 
                u.*,
                COUNT(DISTINCT o.id) as order_count,
                COALESCE(SUM(oi.quantity * oi.price), 0) as total_spend
            FROM users u
            LEFT JOIN orders o ON u.id = o.user_id
            LEFT JOIN order_items oi ON o.id = oi.order_id
            GROUP BY u.id
        ''')
        return [dict(row) for row in cursor.fetchall()]

class SQLiteOrderRepository(Repository[Order]):
    """SQLite implementation of order repository."""
    
    def __init__(self, connection: sqlite3.Connection):
        self.conn = connection
        self.conn.row_factory = sqlite3.Row
    
    def get_by_id(self, order_id: int) -> Optional[Order]:
        # Get order
        cursor = self.conn.execute(
            'SELECT * FROM orders WHERE id = ?', (order_id,)
        )
        row = cursor.fetchone()
        if not row:
            return None
        
        # Get items
        items_cursor = self.conn.execute(
            'SELECT * FROM order_items WHERE order_id = ?', (order_id,)
        )
        items = [
            OrderItem(
                id=i['id'],
                order_id=i['order_id'],
                product_name=i['product_name'],
                quantity=i['quantity'],
                price=i['price']
            )
            for i in items_cursor.fetchall()
        ]
        
        return Order(
            id=row['id'],
            user_id=row['user_id'],
            order_date=row['order_date'],
            status=row['status'],
            items=items
        )
    
    def save(self, order: Order) -> Order:
        if order.id is None:
            # Create new order
            cursor = self.conn.execute(
                'INSERT INTO orders (user_id, order_date, status) VALUES (?, ?, ?)',
                (order.user_id, order.order_date, order.status)
            )
            order.id = cursor.lastrowid
            
            # Add items
            for item in order.items:
                item.order_id = order.id
                item_cursor = self.conn.execute(
                    '''INSERT INTO order_items 
                       (order_id, product_name, quantity, price) VALUES (?, ?, ?, ?)''',
                    (item.order_id, item.product_name, item.quantity, item.price)
                )
                item.id = item_cursor.lastrowid
        else:
            # Update existing
            self.conn.execute(
                'UPDATE orders SET status = ? WHERE id = ?',
                (order.status, order.id)
            )
        
        self.conn.commit()
        return order

# ═══════════════════════════════════════════════════════════════════
# ✨ Service Layer (Business Logic)
# ═══════════════════════════════════════════════════════════════════

class OrderService:
    """Service layer for order operations."""
    
    def __init__(self, user_repo: SQLiteUserRepository, order_repo: SQLiteOrderRepository):
        self.user_repo = user_repo
        self.order_repo = order_repo
    
    def place_order(self, user_email: str, items: List[dict]) -> Order:
        """Place an order for a user."""
        # Find or create user
        user = self.user_repo.get_by_email(user_email)
        if not user:
            raise ValueError(f"User not found: {user_email}")
        
        # Create order with items
        order = Order(
            id=None,
            user_id=user.id,
            order_date=datetime.now().strftime('%Y-%m-%d'),
            status='pending',
            items=[
                OrderItem(None, 0, i['name'], i['qty'], i['price'])
                for i in items
            ]
        )
        
        return self.order_repo.save(order)

# ═══════════════════════════════════════════════════════════════════
# ✨ Demonstration
# ═══════════════════════════════════════════════════════════════════

print("✨ PYTHONIC: Enterprise Data Access Layer\n")

# Initialize repositories
user_repo = SQLiteUserRepository(conn)
order_repo = SQLiteOrderRepository(conn)
order_service = OrderService(user_repo, order_repo)

# Get all users with stats
print("👥 All Users with Statistics:")
print("-" * 60)
for stats in user_repo.get_all_with_stats():
    print(f"  {stats['name']}: {stats['order_count']} orders, "
          f"${stats['total_spend']:.2f} spent")

# Get a specific order with items
print("\n📦 Order Details (ID: 1):")
print("-" * 60)
order = order_repo.get_by_id(1)
if order:
    print(f"  Order #{order.id} - {order.status}")
    print(f"  Date: {order.order_date}")
    print(f"  Items:")
    for item in order.items:
        print(f"    - {item.product_name}: {item.quantity} x ${item.price:.2f} = ${item.total:.2f}")
    print(f"  Total: ${order.total:.2f}")

# Place a new order using the service
print("\n🛒 Placing New Order:")
print("-" * 60)
new_order = order_service.place_order(
    'bob@email.com',
    [
        {'name': 'Premium Subscription', 'qty': 1, 'price': 99.99},
        {'name': 'Extra Storage', 'qty': 2, 'price': 4.99}
    ]
)
print(f"  ✅ Order #{new_order.id} created!")
print(f"  Total: ${new_order.total:.2f}")

---

## 🎯 Activity 3: Build an `add_order` Function

### Your Task:

Implement `add_order(name, items)` that:
1. Finds the user by name (create if not exists)
2. Creates an order for that user
3. Adds all items to the order
4. Returns the updated total spend for that user

### Requirements:
- Use parameterized queries
- Handle user creation if needed
- Use transactions for atomicity
- Return the user's new total spend

### 💡 Hints:

```python
def add_order(name: str, items: List[dict]) -> float:
    # 1. Find or create user
    # 2. Create order
    # 3. Add items
    # 4. Calculate and return total spend
    pass
```

In [ ]:
# ╔══════════════════════════════════════════════════════════════════╗
# ║  👩‍💻 YOUR WORKSPACE: add_order Function                           ║
# ╚══════════════════════════════════════════════════════════════════╝
# Implement a function to add orders for users

from typing import List

# ═══════════════════════════════════════════════════════════════════
# 📝 YOUR CODE HERE
# ═══════════════════════════════════════════════════════════════════

def add_order_attempt(name: str, items: List[dict]) -> float:
    """
    Add an order for a user.
    
    Args:
        name: User's name
        items: List of {'name': str, 'qty': int, 'price': float}
        
    Returns:
        Updated total spend for the user
    """
    # TODO: Implement this function
    # 1. Find user or create if not exists
    # 2. Create new order
    # 3. Add items to order
    # 4. Return total spend
    pass

# ═══════════════════════════════════════════════════════════════════
# 💡 SOLUTION
# ═══════════════════════════════════════════════════════════════════

def add_order(name: str, items: List[dict]) -> float:
    """
    Add an order for a user and return their updated total spend.
    
    Args:
        name: User's name
        items: List of {'name': str, 'qty': int, 'price': float}
        
    Returns:
        Updated total spend for the user
    """
    # Step 1: Find or create user
    cur.execute('SELECT id FROM users WHERE name = ?', (name,))
    row = cur.fetchone()
    
    if row:
        user_id = row['id']
    else:
        # Create new user
        cur.execute(
            'INSERT INTO users (name, email) VALUES (?, ?)',
            (name, f'{name.lower().replace(" ", ".")}@email.com')
        )
        user_id = cur.lastrowid
    
    # Step 2: Create order
    cur.execute('INSERT INTO orders (user_id) VALUES (?)', (user_id,))
    order_id = cur.lastrowid
    
    # Step 3: Add items
    for item in items:
        cur.execute(
            '''INSERT INTO order_items (order_id, product_name, quantity, price)
               VALUES (?, ?, ?, ?)''',
            (order_id, item['name'], item['qty'], item['price'])
        )
    
    conn.commit()
    
    # Step 4: Calculate and return total spend
    cur.execute('''
        SELECT COALESCE(SUM(oi.quantity * oi.price), 0) as total
        FROM orders o
        JOIN order_items oi ON o.id = oi.order_id
        WHERE o.user_id = ?
    ''', (user_id,))
    
    return cur.fetchone()['total']

# ═══════════════════════════════════════════════════════════════════
# Test the solution
# ═══════════════════════════════════════════════════════════════════

print("✅ Testing add_order function:\n")

# Test with existing user
ana_spend = add_order('Ana', [
    {'name': 'Coffee Maker', 'qty': 1, 'price': 79.99},
    {'name': 'Coffee Beans', 'qty': 2, 'price': 15.99}
])
print(f"Ana's updated total spend: ${ana_spend:.2f}")

# Test with new user
new_spend = add_order('Eve', [
    {'name': 'Tablet', 'qty': 1, 'price': 299.99}
])
print(f"Eve's total spend (new user): ${new_spend:.2f}")

---

## 📚 Session Recap

### What We Learned Today:

| Topic | Key Concept | Best Practice |
|-------|-------------|---------------|
| **N+1 Problem** | Query in loops | Use JOINs instead |
| **Subqueries** | Query within query | For complex filtering |
| **CTEs** | WITH clause | Readable, step-by-step |
| **Indexes** | Speed up queries | On WHERE/JOIN columns |
| **Repository Pattern** | Separate data access | Testable, maintainable |

### SQL Query Patterns:

```sql
-- Subquery (filter by aggregate)
SELECT * FROM users 
WHERE id IN (SELECT user_id FROM orders GROUP BY user_id HAVING COUNT(*) > 5);

-- CTE (readable multi-step)
WITH user_totals AS (...)
SELECT * FROM user_totals WHERE total > 100;

-- Window function (ranking)
SELECT *, RANK() OVER (ORDER BY total DESC) as rank FROM orders;
```

### The Evolution of Our Code:

| Level | Data Access Pattern |
|-------|---------------------|
| 🚫 **Bad** | SQL scattered everywhere, N+1 queries |
| 🌱 **Novice** | Functions wrapping SQL |
| 🔧 **Intermediate** | Repository classes |
| ✨ **Pythonic** | Domain models + services + type hints |

### 🎯 Key Takeaways:

1. **Avoid N+1 queries** - Use JOINs instead of loops
2. **Use CTEs** for complex queries - More readable than nested subqueries
3. **Index strategically** - WHERE and JOIN columns benefit most
4. **Repository pattern** - Separates data access from business logic
5. **Type hints** - Make your data layer self-documenting

### 🔜 Next Steps:
- ORM libraries (SQLAlchemy)
- Database migrations
- Connection pooling
- Caching strategies